<a href="https://colab.research.google.com/github/SarthakNarayan/DL-and-ML/blob/master/googlecolab/PyTorchFMNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
import time

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [0]:
# Loading the data
transform = transforms.Compose(
    [transforms.ToTensor()])

trainset = torchvision.datasets.FashionMNIST(root='./data', train=True,
                                        download=True, transform=transform)

testset = torchvision.datasets.FashionMNIST(root='./data', train=False,
                                       download=True, transform=transform)

# classes = ('plane', 'car', 'bird', 'cat',
#            'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [0]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

In [47]:
# Model Creation
# Input size of the image is 32*32
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.conv1 = nn.Conv2d(1, 6, 5)
    # First is the kernel size then stride in MaxPool2d
    # reduces height and width by 2
    # We use thr=e same MaxPool2d layer twice
    self.pool = nn.MaxPool2d(2, 2)
    self.conv2 = nn.Conv2d(6, 16, 5)
    self.fc1 = nn.Linear(16 * 4 * 4, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)

  def forward(self, x):
      x = self.pool(F.relu(self.conv1(x)))
      x = self.pool(F.relu(self.conv2(x)))
      x = x.view(-1, 16 * 4 * 4)
      x = F.relu(self.fc1(x))
      x = F.relu(self.fc2(x))
      x = self.fc3(x)
      return x

net = Net()
# changes for cuda
net.to(device)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=256, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [0]:
# Training
num_epochs = 10
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

start_time = time.time()
for epoch in range(num_epochs):
  running_loss = 0.0
  for i,data in enumerate(trainloader):
    # Changes for cuda
    images , labels = data
    images = images.to(device)
    labels = labels.to(device)
    optimizer.zero_grad()
    output = net(images)
    loss = criterion(output , labels)
    loss.backward()
    optimizer.step()
    
    # Printing stats
    running_loss += loss.item()
    if i % 1000 == 999:    # print every 2000 mini-batches
      print('[%d, %5d] loss: %.10f' % (epoch + 1, i + 1, running_loss / 1000.0))
      running_loss = 0.0
end_time = time.time()
total_time = end_time - start_time
print("Total time taken : {}".format(total_time))

In [53]:
# Testing
test_images , test_labels = next(iter(testloader))
# For cuda 
test_images = test_images.to(device)
test_labels = test_labels.to(device)
test_output = net(test_images)
values , prediction_idx = torch.max(test_output , 1)
print("Expected values")
print(test_labels)
print(prediction_idx)

Expected values
tensor([9, 2, 1, 1], device='cuda:0')
tensor([9, 2, 1, 1], device='cuda:0')


In [54]:
# Getting the accuracy
correct = 0
total = 0

start_times = time.time()
with torch.no_grad():
  for data in testloader:
    test_images , test_labels = data
    test_images = test_images.to(device)
    test_labels = test_labels.to(device)
    output = net(test_images)
    _ , predictions_idx = torch.max(output , 1)
    total = total + len(test_images)
    
    for i in range(len(test_images)):
      if(test_labels[i].item() == predictions_idx[i].item()):
        correct = correct + 1

print(total)
print(correct)
end_times = time.time()
total_times = end_times - start_times
print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))
print("Total time taken : {}".format(total_times))

10000
8312
Accuracy of the network on the 10000 test images: 83 %
Total time taken : 6.193043231964111


In [55]:
# saving the model
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# To save
# Only saves the weights and the network
torch.save(net.state_dict(), '/content/gdrive/My Drive/Colab Notebooks/net.pth')

In [60]:
# To load
# we have to load the model first since it doesnot save the model
net_loaded = Net()
net_loaded.load_state_dict(torch.load('/content/gdrive/My Drive/Colab Notebooks/net.pth')) 
net_loaded.eval()
print("model loaded")

model loaded


In [61]:
test_images , test_labels = next(iter(testloader))
# For cuda 
test_images = test_images.to(device)
test_labels = test_labels.to(device)
test_output = net_loaded(test_images)
values , prediction_idx = torch.max(test_output , 1)
print("Expected values")
print(test_labels)
print(prediction_idx)

RuntimeError: ignored